# Corpus compilation
- text type and genres
- characteristics according to task

In [1]:
# Requests in python
import urllib3
# Regular expression library
import re
# Transform html into a tree im memory
from bs4 import BeautifulSoup
import os
# Get just the text in a html document
import justext

In [39]:
# Disable https warning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# Pass as a browser 
user_agent = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'}

# Pool of boots to make requests
http = urllib3.PoolManager(10, headers=user_agent)


class Crawler:
    
    def __init__(self, corpus_path, max_files, seed_url, url_pattern):
        self.corpus_path = corpus_path # corpus address
        self.max_files = max_files # max amount of sizes
        self.seed_url = seed_url # root url
        self.url_pattern = url_pattern # Select links of interest
        self.visited_links = {} # Hash to store viseted links
        self.to_be_visited = [] # List of link
        
        # If path not exists create it
        if not os.path.exists(self.corpus_path):
            os.makedirs(self.corpus_path)
        
    def crawl(self):
        first_urls = self.get_page(self.seed_url)
        self.add_links(first_urls)
        next_link = self.get_next_link()
        
        file_counter = 1
        while next_link and file_counter < self.max_files:
            links = self.get_page(next_link)
            self.add_links(links)
            next_link = self.get_next_link()
            file_counter += 1
        
    
    def get_page(self, url):
        print("getting page {}".format(url))
        response = http.request('GET', url)

        # store text content
        paragraphs = justext.justext(response.data, justext.get_stoplist("Portuguese"))
        with open("{}/{}.txt".format(self.corpus_path, url.replace(".", "_").replace("/","-")), "w") as output_file:
            for paragraph in paragraphs:
                # Boilerplate is everthing that is not the main text
                if not paragraph.is_boilerplate:
                    output_file.write(paragraph.text)
        
        # get links
        soup = BeautifulSoup(response.data, 'html.parser')
#         links_1 = re.search(self.url_pattern, str(response.data))
#         print(links_1)
#         links = []
#         for group in links_1.groups():
#             links.append("https://linuxdicasesuporte.blogspot.com/2019/{}".format(group))
            
        
        links = [link.get('href') for link in soup.findAll('a', attrs={'href': re.compile(self.url_pattern)})]
        print(links)
        return links

    def add_links(self, links):
        links = list(set(links))
        self.to_be_visited.extend([link for link in links if link not in self.visited_links])

    def get_next_link(self):
        next_link = self.to_be_visited.pop(0)
        self.visited_links[next_link] = None
        return next_link

In [42]:
crawler_seguranca = Crawler("data/corpora/sensacionalista", 50, 
                             "https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/", 
                            "^https://www\.sensacionalista\.com\.br/\d+")

crawler_educacao = Crawler("data/corpora/educacao", 100,
                          "https://g1.globo.com/educacao/noticia/2019/04/12/lacunas-da-politica-nacional-de-alfabetizacao-fazem-especialistas-acenderem-luz-amarela-de-alerta.ghtml",
                          "^https://g1\.globo\.com/educacao/")

In [43]:
crawler_seguranca.crawl()
crawler_educacao.crawl()

getting page https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/
['https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/', 'https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/', 'https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/', 'https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/', 'https

['https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/', 'https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/', 'https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/', 'https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/', 'https://www.sensacionalista.com.br/2019/04/08/alunos-terao-que-entoar-cancao-da-despedida-para-velez-em-todas-as-escolas/', 'https://ww

['https://www.sensacionalista.com.br/2019/04/05/governo-e-tao-atrasado-que-nem-os-relogios-vao-adiantar-mais-diz-analista/', 'https://www.sensacionalista.com.br/2019/04/08/alunos-terao-que-entoar-cancao-da-despedida-para-velez-em-todas-as-escolas/', 'https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/', 'https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/', 'https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/', 'https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/', 'https://www.sensaci

['https://www.sensacionalista.com.br/2019/04/04/reforma-da-previdencia-faz-tigrao-e-tchutchuca-voltarem-a-trabalhar/', 'https://www.sensacionalista.com.br/2019/04/05/governo-e-tao-atrasado-que-nem-os-relogios-vao-adiantar-mais-diz-analista/', 'https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/', 'https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/', 'https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/', 'https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/', 'https://www.sensacionalist

['https://www.sensacionalista.com.br/2019/04/05/governo-e-tao-atrasado-que-nem-os-relogios-vao-adiantar-mais-diz-analista/', 'https://www.sensacionalista.com.br/2019/04/08/alunos-terao-que-entoar-cancao-da-despedida-para-velez-em-todas-as-escolas/', 'https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/', 'https://www.sensacionalista.com.br/2019/04/09/prefeito-crivella-e-substituido-por-um-cone-da-cet-rio-e-ninguem-nota/', 'https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/09/crivella-pede-que-cariocas-nao-temam-enchentes-e-juntem-um-casal-de-cada-especie-2/', 'https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/', 'https://www.sensacionalista.com.br/2019/04/08/jet-ski-por-aplicativo-faz-sucesso-e-desbanca-patinete-no-rio/', 'https://www.sensaci

...

['https://g1.globo.com/educacao/noticia/2019/03/25/secretaria-de-educacao-basica-do-mec-pede-demissao.ghtml', 'https://g1.globo.com/educacao/noticia/2019/03/12/ministro-da-educacao-anuncia-demissao-de-secretario-executivo-da-pasta-rubens-barreto-assume.ghtml', 'https://g1.globo.com/educacao/noticia/2019/03/22/anunciada-como-numero-dois-do-mec-iolene-lima-diz-que-nao-seguira-no-cargo.ghtml', 'https://g1.globo.com/educacao/noticia/2019/03/22/anunciada-como-numero-dois-do-mec-iolene-lima-diz-que-nao-seguira-no-cargo.ghtml']
getting page https://g1.globo.com/educacao/noticia/antecipacao-da-alfabetizacao-na-base-curricular-levanta-debate-sobre-o-ensino-infantil.ghtml
[]
getting page https://g1.globo.com/educacao/noticia/base-nacional-comum-curricular-bncc-e-aprovada-em-conselho.ghtml
['https://g1.globo.com/educacao/noticia/entidades-pedem-suspensao-da-votacao-da-base-nacional-comum-curricular-em-conselho.ghtml', 'https://g1.globo.com/educacao/noticia/justica-determina-que-mec-defina-e-imple

['https://g1.globo.com/educacao/noticia/2019/03/11/governo-federal-exonera-chefe-de-gabinete-secretario-adjunto-assessor-especial-e-tres-diretores-do-mec.ghtml', 'https://g1.globo.com/educacao/noticia/2019/02/05/declaracoes-do-ministro-da-educacao-causam-polemica-e-provocam-reacoes-no-congresso.ghtml', 'https://g1.globo.com/educacao/noticia/2019/02/25/mec-envia-carta-as-escolas-pedindo-que-criancas-sejam-filmadas-durante-execucao-do-hino-nacional.ghtml', 'https://g1.globo.com/educacao/noticia/2019/02/05/declaracoes-do-ministro-da-educacao-causam-polemica-e-provocam-reacoes-no-congresso.ghtml', 'https://g1.globo.com/educacao/noticia/2019/02/26/ministro-da-educacao-diz-ao-stf-ter-sido-infeliz-ao-afirmar-que-brasileiro-e-canibal-no-exterior.ghtml']
getting page https://g1.globo.com/educacao/noticia/2019/03/14/mec-anuncia-segunda-mudanca-na-secretaria-executiva-em-tres-dias.ghtml
['https://g1.globo.com/educacao/noticia/2019/02/26/ministro-da-educacao-diz-ao-stf-ter-sido-infeliz-ao-afirmar-

['https://g1.globo.com/educacao/noticia/2019/02/25/mec-envia-carta-as-escolas-pedindo-que-criancas-sejam-filmadas-durante-execucao-do-hino-nacional.ghtml']
getting page https://g1.globo.com/educacao/noticia/2019/02/25/mec-envia-carta-as-escolas-pedindo-que-criancas-sejam-filmadas-durante-execucao-do-hino-nacional.ghtml
['https://g1.globo.com/educacao/noticia/2019/02/25/juristas-e-educadores-criticam-pedido-do-mec-para-execucao-de-hino-e-leitura-de-carta-com-slogan-de-bolsonaro.ghtml', 'https://g1.globo.com/educacao/noticia/2019/02/25/juristas-e-educadores-criticam-pedido-do-mec-para-execucao-de-hino-e-leitura-de-carta-com-slogan-de-bolsonaro.ghtml']
getting page https://g1.globo.com/educacao/noticia/2019/02/25/mec-envia-carta-as-escolas-pedindo-que-criancas-sejam-filmadas-durante-execucao-do-hino-nacional.ghtml
['https://g1.globo.com/educacao/noticia/2019/02/25/juristas-e-educadores-criticam-pedido-do-mec-para-execucao-de-hino-e-leitura-de-carta-com-slogan-de-bolsonaro.ghtml', 'https:/

[]
getting page https://g1.globo.com/educacao/noticia/nao-faz-sentido-esperar-ate-os-9-anos-para-alfabetizacao-plena-diz-executiva-do-mec-veja-videos.ghtml
[]
getting page https://g1.globo.com/educacao/noticia/2019/02/25/juristas-e-educadores-criticam-pedido-do-mec-para-execucao-de-hino-e-leitura-de-carta-com-slogan-de-bolsonaro.ghtml
[]
getting page https://g1.globo.com/educacao/noticia/2019/02/25/juristas-e-educadores-criticam-pedido-do-mec-para-execucao-de-hino-e-leitura-de-carta-com-slogan-de-bolsonaro.ghtml
[]
getting page https://g1.globo.com/educacao/noticia/2019/02/25/juristas-e-educadores-criticam-pedido-do-mec-para-execucao-de-hino-e-leitura-de-carta-com-slogan-de-bolsonaro.ghtml
[]
getting page https://g1.globo.com/educacao/noticia/prova-que-avalia-alunos-do-fim-do-fundamental-vai-incluir-questoes-de-ciencias-humanas-e-da-natureza.ghtml
['https://g1.globo.com/educacao/noticia/prova-brasil-comeca-ser-aplicada-em-todo-o-pais-a-partir-desta-segunda.ghtml', 'https://g1.globo.com

IndexError: pop from empty list